In [ ]:
# Text Emotions Classification

Metin duyguları sınıflandırması, doğal dil işleme ve metin sınıflandırması sorunudur. Burada bir metnin duygusunu sınıflandırmak için bir metin sınıflandırma modeli eğitmemiz gerekiyor.

Bu sorunu çözmek için metinlerin ve duygularının etiketlenmiş verilerine ihtiyacımız var. Kaggle'da bu sorunu çözmek için ideal bir veri kümesi buldum. Veri setini adresinden indirebilirsiniz burada.

[Linktext](https:///www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp

Aşağıdaki bölümde, Makine Öğrenimi ve Python programlama dilini kullanarak Metin Duyguları Sınıflandırma görevi için bir metin sınıflandırma modelinin nasıl eğitileceğini size anlatacağım.

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow

In [4]:
# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [6]:
# Load the data
# The dataset is in a text file with each line containing text and emotion label separated by a semicolon.
# We need to load this into a pandas DataFrame.

data = []
with open('/content/train.txt', 'r') as f:
    for line in f:
        text, emotion = line.strip().split(';')
        data.append({'text': text, 'emotion': emotion})

df = pd.DataFrame(data)

display(df.head())
display(df['emotion'].value_counts())

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


,count
emotion,
joy,5362
sadness,4666
anger,2159
fear,1937
love,1304
surprise,572


In [8]:
# Bu, doğal dil işlemeyle ilgili bir sorun olduğundan, verileri tokenize ederek başlayacağım:

In [10]:
texts = df["text"].tolist()
labels = df["emotion"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Şimdi dizileri bir sinir ağına beslemek için aynı uzunlukta pedlememiz gerekiyor. Metinlerin dizilerini aynı uzunluğa sahip olacak şekilde nasıl doldurabiliriz:

In [11]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

Şimdi sınıfları dizelerden sayısal bir gösterime dönüştürmek için etiket kodlayıcı yöntemini kullanacağım:

In [12]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

Şimdi One-hot etiketlerini kodlamaya gidiyoruz. Bir sıcak kodlama, kategorik etiketlerin, her etiketin tek bir 1 hariç tüm sıfırların vektörü olarak temsil edildiği ikili bir temsile dönüştürülmesini ifade eder. Bu gereklidir, çünkü makine öğrenimi algoritmaları sayısal verilerle çalışır. İşte One-hot etiketlerini şu şekilde kodlayabiliriz:

In [13]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [14]:
# Text Emotions Classification Model

Şimdi verileri eğitim ve test setlerine ayıracağız:

In [15]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

Şimdi sınıflandırma problemimiz için bir sinir ağı mimarisi tanımlayalım ve bunu duyguları sınıflandırmak için bir model eğitmek için kullanalım:

In [16]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 26s 57ms/step - accuracy: 0.3876 - loss: 1.5157 - val_accuracy: 0.7219 - val_loss: 0.8043
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 34s 39ms/step - accuracy: 0.8886 - loss: 0.3724 - val_accuracy: 0.8081 - val_loss: 0.5692
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - accuracy: 0.9876 - loss: 0.0558 - val_accuracy: 0.8091 - val_loss: 0.6122
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 33ms/step - accuracy: 0.9959 - loss: 0.0221 - val_accuracy: 0.8153 - val_loss: 0.6361
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9962 - loss: 0.0193 - val_accuracy: 0.8172 - val_loss: 0.6858
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.9965 - loss: 0.0150 - val_accuracy: 0.8147 - val_loss: 0.7049
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.9971 - loss: 0.0114 - val_accuracy: 0.8094 - val_loss: 0.7398
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - accuracy: 0.9969 - loss: 0.0096 - val_accurac

Şimdi bir cümleyi giriş metni olarak alalım ve modelin nasıl performans gösterdiğini görelim:

In [17]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
['sadness']


Özet
Metin duygu sınıflandırması, metnin arkasındaki bağlamı ve duyguyu anlayarak bir metne duygu atama sorunudur. Gerçek dünyadaki bir örnek, metni anlayarak en alakalı emojiyi öneren bir iPhone'un klavyesidir.